In [3]:
import os
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import pickle
import pandas as pd

In [4]:
enable_prallel_processing = 0 # 0-disable 1-enable
no_sub_to_run_in_parallel = 4
sub_det_df = pd.read_excel('data/rs_fmri_sub_details_WO_AS.xlsx')
cur_sub_list_before = sub_det_df['Subject_ID_before'].to_numpy()
cur_sub_list_after = sub_det_df['Subject_ID_after'].to_numpy()
subject_ids = np.concatenate((cur_sub_list_before,cur_sub_list_after))
print('# of subjects to Process:', len(subject_ids))

# of subjects to Process: 178


In [5]:
data_dir = '/data-braingut/bharath/nifty_used_data'
FS_dir = '/data-braingut/bharath/anat_freesurfer_V6'
if not (os.path.isdir('/data-braingut/bharath/fmri_bandpass')):
    os.mkdir('/data-braingut/bharath/fmri_bandpass')

In [11]:
def run_afni_proc(sub_id,FS_dir, data_dir):
    cmdline = ('afni_proc.py -subj_id %s \
    -blocks despike tshift align volreg blur mask scale regress \
    -copy_anat %s/%s/SUMA/brain.nii.gz \
    -anat_has_skull no \
    -anat_follower anat_w_skull anat %s/%s/SUMA/T1.nii.gz \
    -anat_follower_ROI aaseg anat  %s/%s/SUMA/aparc.a2009s+aseg.nii.gz \
    -anat_follower_ROI aeseg epi  %s/%s/SUMA/aparc.a2009s+aseg.nii.gz \
    -anat_follower_ROI aasegbig anat  %s/%s/SUMA/aparc+aseg.nii.gz \
    -anat_follower_ROI aesegbig epi  %s/%s/SUMA/aparc+aseg.nii.gz \
    -anat_follower_ROI FSvent epi %s/%s/SUMA/fs_ap_latvent.nii.gz \
    -anat_follower_ROI FSWe epi %s/%s/SUMA/fs_ap_wm.nii.gz \
    -anat_follower_erode FSvent FSWe \
    -dsets %s/%s/func/sub-%s_task-rest_acq-TR2890.nii.gz \
    -tcat_remove_first_trs 4 \
    -align_opts_aea -cost lpc+ZZ -giant_move -check_flip \
    -volreg_align_to MIN_OUTLIER \
    -blur_size 4 \
    -mask_epi_anat yes \
    -regress_motion_per_run \
    -regress_bandpass 0.01 0.1 \
    -regress_ROI_PC FSvent 3 \
    -regress_ROI_PC_per_run FSvent \
    -regress_anaticor_fast \
    -regress_anaticor_label FSWe \
    -regress_apply_mot_types demean deriv \
    -regress_est_blur_epits \
    -regress_est_blur_errts \
    -out_dir /data-braingut/bharath/fmri_bandpass/%s \
    -html_review_style pythonic')%(sub_id,FS_dir,sub_id,FS_dir,sub_id,FS_dir,sub_id,FS_dir,sub_id,FS_dir,sub_id,FS_dir,sub_id,FS_dir,sub_id,FS_dir,sub_id,
                                   data_dir,sub_id,sub_id,sub_id)
#     print(cmdline)
    os.system(cmdline)    
    cmdline1 = ('tcsh -xef proc.%s 2>&1 | tee output.proc.%s')%(sub_id,sub_id)
#     print(cmdline1)    
    os.system(cmdline1)

In [ ]:
if(enable_prallel_processing == 1):
    with ProcessPoolExecutor(max_workers = no_sub_to_run_in_parallel) as executor:
        for i in subject_ids:
            executor.submit(run_afni_proc, i, FS_dir, data_dir)  
else:
    for i in subject_ids:
        run_afni_proc(i, FS_dir, data_dir)  

In [ ]:
moveto_dir = '/data-braingut/bharath/fmri_bandpass/output_proc'
if not (os.path.isdir(moveto_dir)):
    os.mkdir(moveto_dir)
for sub_id in subject_ids:
    shutil.move('output.proc.%s'%sub_id, moveto_dir)
    shutil.move('proc.%s'%sub_id, moveto_dir)  